In [2]:
%load_ext autoreload
%autoreload 2

from cmipper import functions_creche, utils, config, parallelised_download_and_process, main, file_ops
import xarray as xa
import re
from pathlib import Path


/maps/rt582/miniforge3/envs/shiftpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:




# def extract_lat_lon_ranges_from_fp(fp: Path | str):
#     # Define the regular expression pattern
#     pattern = re.compile(
#         r".*_[ns](?P<north>-?\d+-?\d)_[ns](?P<south>-?\d+-?\d)_[we](?P<west>-?\d+-?\d)_[we](?P<east>-?\d+-?\d).*\.nc",
#             # r".*_[ns](?P<north>-?\d+-\d+)_[ns](?P<south>-?\d+-\d+)_[we](?P<west>-?\d+-\d+)_[we](?P<east>-?\d+-\d+).*\.nc",
#         re.IGNORECASE,
#     )
#     print(pattern)
#     # Match the pattern in the filename
#     match = pattern.match(str(Path(fp).name))
#     print(match)
#     if match:
#         # Extract latitude and longitude values: if - in string (representing decimal point), convert to decimal point
#         norther = float(match.group("north")) if "-" not in match.group("north") else float(utils.dash_process_coordinate(match.group("north")))
#         souther = float(match.group("south")) if "-" not in match.group("south") else float(utils.dash_process_coordinate(match.group("south")))
#         wester = float(match.group("west")) if "-" not in match.group("west") else float(utils.dash_process_coordinate(match.group("west")))
#         easter = float(match.group("east")) if "-" not in match.group("east") else float(utils.dash_process_coordinate(match.group("east")))

#         # Create lists of latitudes and longitudes
#         lats = [norther, souther]
#         lons = [wester, easter]
#         return lats, lons
#     else:
#         raise ValueError(f"Could not extract latitudes and longitudes from {fp}. Ensure the filename is formatted correctly.")





def round_to_1_sf_after_decimal(num):
    """
    Rounds a number to 1 significant figure after the decimal point.

    Args:
        num (float): The number to be rounded.

    Returns:
        float: The rounded number.

    Examples:
        >>> round_to_1_sf_after_decimal(3.14159)
        3.1
        >>> round_to_1_sf_after_decimal(10.567)
        10.6
        >>> round_to_1_sf_after_decimal(0.005)
        0.0
    """
    parts = str(num).split('.')
    # if there is a decimal part
    if len(parts) == 2:
        dec_part = parts[1]

        # determine number of zeros before first non-zero digit
        num_zeros = 0
        for i in range(len(dec_part)):
            if dec_part[i] == '0':
                num_zeros += 1
            else:
                break
        return float(parts[0] + '.' + num_zeros*'0' + f"{float(f"{float(dec_part):.1g}"):g}"[0])    # noqa
    else:
        return float(parts[0])


def replace_dot_with_dash(string: str) -> str:
    """
    Replace all occurrences of "." with "-" in a string.

    Parameters
    ----------
        string (str): The input string.

    Returns
    -------
        str: The modified string with "." replaced by "-".
    """
    return string.replace(".", "-")


def lat_lon_string_from_tuples(
    lats: tuple[float, float], lons: tuple[float, float], dp: int = 0
):
    min_lat, max_lat = min(lats), max(lats)
    min_lon, max_lon = min(lons), max(lons)

    lats = [round_to_1_sf_after_decimal(min_lat), round_to_1_sf_after_decimal(max_lat)]
    lons = [round_to_1_sf_after_decimal(min_lon), round_to_1_sf_after_decimal(max_lon)]

    lats_strs = [f"n{replace_dot_with_dash(str(abs(lat)))}" if lat > 0 else f"s{replace_dot_with_dash(str(abs(lat)))}" for lat in lats]   # noqa
    lons_strs = [f"w{replace_dot_with_dash(str(abs(lon)))}" if lon < 0 else f"e{replace_dot_with_dash(str(abs(lon)))}" for lon in lons]   # noqa

    return "_".join(lats_strs + lons_strs)

lats = [-20.023323, 4]
lons = [139.0000203, 146.0]


lat_lon_string_from_tuples(lats, lons)

's20-02_n4-0_e139-00002_e146-0'

In [13]:
def replace_dash_with_dot(string: str):
    return string.replace("-", ".")

def replace_dot_with_dash(string: str):
    return string.replace(".", "-")

def replace_decimal_dash_with_dot(string):
    return re.sub(r'(?<=\d)-(?=\d)', '.', string)


def dash_process_coordinate(num_string):
    if num_string[0] == "-":
        # if dash in place of point
        if "-" in num_string[1:] and re.match(r'\d+-?\d?+', num_string[1:]):
            return -float(replace_decimal_dash_with_dot(num_string[1:]))
        else:
            return -float(num_string[1:])
    # Check if the dash is sandwiched between two numbers before replacing it
    if "-" in num_string and re.match(r'\d+-\d+', num_string):
        return float(replace_decimal_dash_with_dot(num_string))
    else:
        return float(num_string)


def extract_lat_lon_ranges_from_fp(fp: Path | str):
    # Define the regular expression pattern
    coord_pattern = re.compile(
        # r".*_[ns](?P<north>-?\d+-?.?\d?)_[ns](?P<south>-?\d+-?.?\d?)_[we](?P<west>-?\d+-?.?\d?)_[we](?P<east>-?\d+\.?\d*).*.nc",
        r".*_[ns](?P<north>-?\d+-?.?\d?)_[ns](?P<south>-?\d+-?.?\d?)_[we](?P<west>-?\d+-?.?\d?)_[we](?P<east>-?\d+-?\.?\d*).*.nc",
        re.IGNORECASE,
    )
    coord_match = coord_pattern.match(str(Path(fp).name))

    # Match the pattern in the filename
    if not coord_match:
        raise ValueError(f"Could not extract latitudes and longitudes from {fp}. Ensure the filename is formatted correctly.")

    # Extract cardinal directions and determine signs
    cardinal_directions = re.findall(r"([nsew])(?=-?\d+)", str(Path(fp).name), re.IGNORECASE)
    direction_signs = [1 if d.lower() in 'ne' else -1 for d in cardinal_directions]

    # Extract and process coordinates
    north = float(dash_process_coordinate(coord_match.group("north")))
    south = float(dash_process_coordinate(coord_match.group("south")))
    west = float(dash_process_coordinate(coord_match.group("west")))
    east = float(dash_process_coordinate(coord_match.group("east")))

    # Apply direction_signs. Technically correct to have e.g. W-130 as 130, but this doesn't fit with old naming
    north *= direction_signs[0]
    south *= direction_signs[1]
    west *= direction_signs[2]
    east *= direction_signs[3]

    # Create lists of latitudes and longitudes
    lats = [north, south]
    lons = [west, east]

    return lats, lons

# fp = "/maps-priv/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/regridded/concatted_vars_N0_S-32_W130_E170/so_N0_S32_W-130_E170.nc"
fp = "/maps-priv/maps/rt582/coralshift/data/bathymetry/gebco/gebco_2023_n0.0_s-40.0_w130.0_e170.0.nc"
# fp = "/maps-priv/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/regridded/concatted_vars_N0_S-32_W130_E170/wfo_N0_S-32_W130_E170_sfc_ll_195000-204912.nc"

extract_lat_lon_ranges_from_fp(fps[0])

([-90.1, 90.1], [-180.1, 180.1])

In [41]:
out = xa.open_dataset("/maps/rt582/coralshift/data/bathymetry/gebco/gebco_2023_n-10.0_s-30.0_w140.0_e160.0.nc")
out.sortby(list(out.dims)) 

<xarray.Dataset> Size: 46MB
Dimensions:    (lat: 4800, lon: 4800)
Coordinates:
  * lat        (lat) float64 38kB -30.0 -29.99 -29.99 ... -10.01 -10.01 -10.0
  * lon        (lon) float64 38kB 140.0 140.0 140.0 140.0 ... 160.0 160.0 160.0
Data variables:
    elevation  (lat, lon) int16 46MB ...
Attributes: (12/36)
    title:                           The GEBCO_2023 Grid - a continuous terra...
    summary:                         The GEBCO_2023 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/f98b053b-0cbc-6c23-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/f98b053b-0cbc-6c23-e053-6c8...
    references:                      DOI: 10.5285/f98b053b-0cbc-6c23-e053-6c8...
    node_offset:                     1.0

In [1]:
fps = ['/maps-priv/maps/rt582/coralshift/data/ground_truth/UNEP_GDCR/rasters/unep_0-004_S90-1_N90-1_W180-1_E180-1.nc', '/maps-priv/maps/rt582/coralshift/data/ground_truth/UNEP_GDCR/rasters/unep_0-004_S5-1_N0-1_E139-9_E145-1.nc']

In [11]:
extract_lat_lon_ranges_from_fp(fps[0])

([-90.1, 90.1], [-180.1, 180.0])

In [10]:
file_ops.find_files_for_area(fps, lat_range=[-90.1,90.1], lon_range=[-180.1,180.1])

[]

In [10]:
def find_intersecting_cmip(
    variables: list[str],
    source_id: str = "EC-Earth3P-HR",
    member_id: str = "r1i1p2f1",
    lats: list[float, float] = [-40, 0],
    lons: list[float, float] = [130, 170],
    year_range: list[int, int] = [1950, 2014],
    levs: list[int, int] = [0, 20],
    cmip6_data_dir: Path | str = None
):

    # check whether exact intersecting cropped file already exists
    cmip6_dir_fp = config.cmip6_data_dir / source_id / member_id / "regridded" if not cmip6_data_dir else Path(cmip6_data_dir)
    print("Searching for correct CMIP files in:", cmip6_dir_fp)
    # TODO: include levs check. Much harder, so leaving for now
    correct_area_fps = list(
        file_ops.find_files_for_area(cmip6_dir_fp.rglob("*.nc"), lat_range=lats, lon_range=lons),
    )
    # check that also spans full year range
    correct_fps = file_ops.find_files_for_time(correct_area_fps, year_range=sorted(year_range))

    if len(correct_fps) > 0:
        # check that file includes all variables in variables list
        for fp in correct_fps:
            if all(variable in str(fp) for variable in variables):
                ds = utils.process_xa_d(xa.open_dataset(fp))
                ds_years = ds["time.year"]
                return (
                    ds.sel(
                        time=(ds_years >= min(year_range))
                        & (ds_years <= max(year_range)),
                        latitude=slice(min(lats), max(lats)),
                        longitude=slice(min(lons), max(lons)),
                    )[variables],
                    fp,
                )
    return None, None


# find_intersecting_cmip(["so"], lats=[-20, 0], lons=[140, 160], year_range=[1950, 2015], cmip6_data_dir=Path("/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/regridded"))
find_intersecting_cmip(["so"], lats=[-20, 0], lons=[140, 160], year_range=[1950, 2015], cmip6_data_dir=Path("/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/regridded/concatted_vars"))

                    

Searching for correct CMIP files in: /maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/regridded/concatted_vars


(None, None)

In [45]:
pattern = re.compile(r"([nsew])(?=-?\d+)", re.IGNORECASE)

match = pattern.findall(fp)

directions = []
for direction in match:
    if direction.lower() == "n" or direction.lower() == "e":
        directions.append(1)
    else:
        directions.append(-1)

directions
        

[-1, -1, -1, 1]

In [ ]:
xa.open_dataset("/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/og_grid/tos/tos_uncropped_sfc_tp_200401-200412.nc")

In [ ]:
xa.open_dataset("/maps-priv/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/newtest/og_grid/mlotst/mlotst_uncropped_sfc_tp_199301-199312.nc")

In [ ]:
parallelised_download_and_process.delete_corrupt_files("EC-Earth3P-HR", "r1i1p2f1")

In [ ]:
main.main()